In [1]:
from tqdm import tqdm
import math
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
import json

f = open("./NLP2590/labeled_news/data.json")
data = json.load(f)
f.close()

In [5]:
import spacy
sp = spacy.load('en_core_web_sm')

In [6]:
paragraphs = []

for news in data:
    label = news['label']
    body = news['body']
    
    paragraphs.append([body.strip(), label])
    
#     for p in body.split("\n"):
#         if p.strip() == "":
#             continue
            
#         paragraphs.append([p.strip(), label])

In [85]:
len(paragraphs)

714

In [8]:
paragraphs[0]

['Alaina Lattin was served an eviction notice last month at her home in Conroe, Texas.\n"It was a very loud banging," Lattin, 32, said. "They just kept screaming, \'Leasing office! Leasing office!\'" Now, the single mother is worried that she and her four children will become homeless.\nLattin is one of millions of Americans facing eviction amid the coronavirus pandemic. Even as unemployment levels remain at historic highs and cases of the virus continue to surge, the federal eviction moratorium in the first stimulus package was allowed to expire at the end of last month.\nPresident Donald Trump said the executive action he took over the weekend would help keep renters in their homes, but advocates say the move won\'t actually do much .\n"It creates the impression that something was done when, in fact, nothing was done," said John Pollock , coordinator of the National Coalition for a Civil Right to Counsel.\nMore from Personal Finance:\nTrump wants stimulus checks to be more than $1,20

In [7]:
import re

punctuation = re.compile('[^\w\s]')
space = re.compile('  +')

In [87]:
p = space.sub(' ', punctuation.sub('', paragraphs[0][0]))
p

'Alaina Lattin was served an eviction notice last month at her home in Conroe Texas\nIt was a very loud banging Lattin 32 said They just kept screaming Leasing office Leasing office Now the single mother is worried that she and her four children will become homeless\nLattin is one of millions of Americans facing eviction amid the coronavirus pandemic Even as unemployment levels remain at historic highs and cases of the virus continue to surge the federal eviction moratorium in the first stimulus package was allowed to expire at the end of last month\nPresident Donald Trump said the executive action he took over the weekend would help keep renters in their homes but advocates say the move wont actually do much \nIt creates the impression that something was done when in fact nothing was done said John Pollock coordinator of the National Coalition for a Civil Right to Counsel\nMore from Personal Finance\nTrump wants stimulus checks to be more than 1200\nHow HEALS Act stimulus checks would

In [88]:
sentence = sp(p)

In [89]:
sentence

Alaina Lattin was served an eviction notice last month at her home in Conroe Texas
It was a very loud banging Lattin 32 said They just kept screaming Leasing office Leasing office Now the single mother is worried that she and her four children will become homeless
Lattin is one of millions of Americans facing eviction amid the coronavirus pandemic Even as unemployment levels remain at historic highs and cases of the virus continue to surge the federal eviction moratorium in the first stimulus package was allowed to expire at the end of last month
President Donald Trump said the executive action he took over the weekend would help keep renters in their homes but advocates say the move wont actually do much 
It creates the impression that something was done when in fact nothing was done said John Pollock coordinator of the National Coalition for a Civil Right to Counsel
More from Personal Finance
Trump wants stimulus checks to be more than 1200
How HEALS Act stimulus checks would be diff

In [90]:
for word in sentence:
    print(word.text,  word.lemma_)

Alaina Alaina
Lattin Lattin
was be
served serve
an an
eviction eviction
notice notice
last last
month month
at at
her -PRON-
home home
in in
Conroe Conroe
Texas Texas

 

It -PRON-
was be
a a
very very
loud loud
banging bang
Lattin Lattin
32 32
said say
They -PRON-
just just
kept keep
screaming scream
Leasing Leasing
office office
Leasing leasing
office office
Now now
the the
single single
mother mother
is be
worried worried
that that
she -PRON-
and and
her -PRON-
four four
children child
will will
become become
homeless homeless

 

Lattin Lattin
is be
one one
of of
millions million
of of
Americans Americans
facing face
eviction eviction
amid amid
the the
coronavirus coronavirus
pandemic pandemic
Even even
as as
unemployment unemployment
levels level
remain remain
at at
historic historic
highs high
and and
cases case
of of
the the
virus virus
continue continue
to to
surge surge
the the
federal federal
eviction eviction
moratorium moratorium
in in
the the
first first
stimulus stimulus


In [8]:
paragraphs_lemmaed = []

for par in tqdm(paragraphs):
    p = space.sub(' ', punctuation.sub('', par[0]))
    s = sp(p)
    
    words = []
    
    for w in s:
        words.append(w.lemma_.lower())
        
    paragraphs_lemmaed.append([" ".join(words), par[1]])
    
paragraphs_lemmaed[0]

100%|██████████| 714/714 [00:51<00:00, 13.76it/s]


['alaina lattin be serve an eviction notice last month at -pron- home in conroe texas \n -pron- be a very loud bang lattin 32 say -pron- just keep scream leasing office leasing office now the single mother be worried that -pron- and -pron- four child will become homeless \n lattin be one of million of americans face eviction amid the coronavirus pandemic even as unemployment level remain at historic high and case of the virus continue to surge the federal eviction moratorium in the first stimulus package be allow to expire at the end of last month \n president donald trump say the executive action -pron- take over the weekend would help keep renter in -pron- home but advocate say the move will not actually do much \n -pron- create the impression that something be do when in fact nothing be do say john pollock coordinator of the national coalition for a civil right to counsel \n more from personal finance \n trump want stimulus check to be more than 1200 \n how heals act stimulus check 

In [9]:
words = set()

for par in tqdm(paragraphs_lemmaed):
    for w in par[0].split():
        words.add(w)

words = list(words)
        
len(words)

100%|██████████| 714/714 [00:00<00:00, 6002.53it/s]


24181

In [99]:
# word_paragraph_count = {}

# for word in tqdm(words):
#     word_paragraph_count[word] = 0
#     for par in paragraphs_lemmaed:
#         if word in par[0].split():
#             word_paragraph_count[word] += 1

In [10]:
class_paragraph_count = {-1: 0, 0: 0, 1: 0}

for par in tqdm(paragraphs_lemmaed):
    class_paragraph_count[par[1]] += 1
    
class_paragraph_count

100%|██████████| 714/714 [00:00<00:00, 737618.98it/s]


{-1: 374, 0: 159, 1: 181}

In [25]:
word_class_paragraph_count = {}

for word in tqdm(words):
    word_class_paragraph_count[word] = {-1: 0, 0: 0, 1: 0}
    for par in paragraphs_lemmaed:
        if word in par[0].split():
            word_class_paragraph_count[word][par[1]] += 1

100%|██████████| 24181/24181 [13:09<00:00, 30.61it/s]


In [26]:
list(word_class_paragraph_count.items())[:10]

[('firstofitskind', {-1: 0, 0: 0, 1: 1}),
 ('churches', {-1: 1, 0: 0, 1: 0}),
 ('chilean', {-1: 1, 0: 0, 1: 0}),
 ('immediate', {-1: 15, 0: 8, 1: 8}),
 ('responsible', {-1: 16, 0: 8, 1: 5}),
 ('olaf', {-1: 1, 0: 0, 1: 0}),
 ('cosme', {-1: 1, 0: 0, 1: 0}),
 ('revoke', {-1: 2, 0: 2, 1: 0}),
 ('descend', {-1: 2, 0: 0, 1: 0}),
 ('nigro', {-1: 1, 0: 0, 1: 0})]

In [27]:
word_paragraph_count = {}
for k, v in word_class_paragraph_count.items():
    word_paragraph_count[k] = sum(v.values())

In [28]:
total = len(paragraphs_lemmaed)
total

714

In [29]:
pmi = {}

# p(w|c)
for word in tqdm(words):
    pmi[word] = {}
    
    if word_class_paragraph_count[word][-1] != 0:
        pmi[word][-1] = math.log(
            (word_class_paragraph_count[word][-1] / total) / 
            (class_paragraph_count[-1] / total))
    else:
        pmi[word][-1] = math.log(
            (1 / total) / 
            (class_paragraph_count[-1] / total))
        
    if word_class_paragraph_count[word][1] != 0:
        pmi[word][1] = math.log(
            (word_class_paragraph_count[word][1] / total) / 
            (class_paragraph_count[1] / total))
    else:
        pmi[word][1] = math.log(
            (1 / total) / 
            (class_paragraph_count[1] / total))
        

# # p(c|w)
# for word in tqdm(words):
#     pmi[word] = {}
    
#     if word_class_paragraph_count[word][-1] != 0:
#         pmi[word][-1] = math.log(
#             (word_class_paragraph_count[word][-1] / total))
#     else:
#         pmi[word][-1] = math.log(
#             (1 / total))
        
#     if word_class_paragraph_count[word][1] != 0:
#         pmi[word][1] = math.log(
#             (word_class_paragraph_count[word][1] / total))
#     else:
#         pmi[word][1] = math.log(
#             (1 / total))

100%|██████████| 24181/24181 [00:00<00:00, 434700.00it/s]


In [30]:
S = {}

for word, values in pmi.items():
    S[word] = values[1] - values[-1]

In [31]:
s_df = pd.DataFrame.from_dict(S, orient='index')
s_df.columns = ['sent']

In [32]:
freq = pd.DataFrame.from_dict(word_paragraph_count, orient='index')
freq.columns = ['freq']

In [33]:
s_df = s_df.join(freq)

In [34]:
s_df.sort_values('sent', ascending=False)

,sent,freq
prnewswire,3.616131,24
multimedia,3.433809,20
proud,2.805200,10
innovative,2.805200,11
fun,2.671669,9
interesting,2.671669,9
integrate,2.671669,8
download,2.622879,33
plasma,2.517518,8
smooth,2.517518,7


In [35]:
s_df.loc['negative']

sent    -0.487264
freq    66.000000
Name: negative, dtype: float64

In [25]:
# f = open("pmi.json", "w")
# json.dump(S, f)
# f.close()

# Plotting

In [21]:
vader_lexicon_file = './data/vader_lexicon.txt'

In [22]:
vader_lexicon = {}
with open(vader_lexicon_file, 'r') as f:
    for line in f.readlines():
        line = line.strip()
        vader_lexicon[line.split('\t')[0]] = float(line.split('\t')[1])

In [23]:
vader_words = list(vader_lexicon.keys())

In [36]:
s_new = s_df[~s_df.index.isin(vader_words)].copy()
s_com = s_df[s_df.index.isin(vader_words)].copy()

### Common word modification

In [37]:
s_com['vader'] = s_com.apply(lambda x: vader_lexicon.get(x.name), axis=1)

In [38]:
s_com.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1522 entries, responsible to intelligent
Data columns (total 3 columns):
sent     1522 non-null float64
freq     1522 non-null int64
vader    1522 non-null float64
dtypes: float64(2), int64(1)
memory usage: 47.6+ KB


In [39]:
s_com_filter = s_com[s_com['freq']>100].copy().reset_index()

In [45]:
px.scatter(s_com_filter, x='vader', y='sent', text='index', size='freq',
           labels={'vader': 'Vader score', 'sent': 'Customized score'}, 
           title='Comparison of Lexicon')

### New word add-in

In [41]:
s_new.loc['outbreak']

sent     -0.466785
freq    232.000000
Name: outbreak, dtype: float64

In [42]:
thres = 150
s_new_filter = s_new[s_new['freq']>thres].copy().reset_index()

In [43]:
s_new_filter

,index,sent,freq
0,standards,-0.082901,192
1,editing,-0.105010,239
2,could,-0.083727,325
3,company,0.310815,219
4,be,0.002348,702
5,covid19,0.037478,394
6,down,-0.271757,243
7,even,-0.246702,214
8,thomson,-0.102121,206
9,virus,-0.379243,257


In [46]:
px.scatter(s_new_filter, x='freq', y='sent', text='index')

### Update dictionary 

In [209]:
update_dict = {
    # existing
    "crisis": 0,
    "positive": -1,
    "great": 0,
    "authority": -2,
    
    "Crisis": 0,
    "Positive": -1,
    "Great": 0,
    "Authority": -2,
    
    # new
    "coronavirus": -1,
    "pandemic": 0.5,
    "outbreak": -1.5,
    "virus": -1,
    "lockdown": -1,
    "trump": -1.5,
    
    "Coronavirus": -1,
    "Pandemic": 0.5,
    "Outbreak": -1.5,
    "Virus": -1,
    "Lockdown": -1,
    "Trump": -1.5,
    "TRUMP": -1.5,
}

In [210]:
f = open("update_dict.json", "w")
json.dump(update_dict, f)
f.close()